In [15]:
from re import S
from fastapi.params import Query
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from dotenv import load_dotenv
import os, sys, json, time

In [11]:
pip install googleapiclient, google-api-python-client

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement googleapiclient (from versions: none)
ERROR: No matching distribution found for googleapiclient

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import json
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

def update_info_in_sheets():
    """
    Updates the text in cell H1 to 'music info' for each user's Google Sheet.
    """
    with open("db/main/db.json", "r", encoding="utf-8") as file:
        table_list = json.load(file)

    for index, account in enumerate(table_list["accounts"]):
        if index > 14:
            try:
                # Set up Google Sheets API credentials
                scope = [
                    "https://spreadsheets.google.com/feeds",
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive.file",
                    "https://www.googleapis.com/auth/drive"
                ]
                credentials = ServiceAccountCredentials.from_json_keyfile_name(
                    'static/reelstranscription-a94a4b07252e.json', scope)
                client = gspread.authorize(credentials)

                # Open the Google Sheet
                google_sheet_url = f'https://docs.google.com/spreadsheets/d/{account["table_id"]}/edit?usp=sharing'


                list_array = ['DATA', 'DATA_TIKTOK']
                for list in list_array:
                    # sheet = client.open_by_url(google_sheet_url).worksheet(list)
                    # # Получение текущих значений из столбца D, начиная с D2
                    # current_values = sheet.get('D2:D')  # Получаем все значения начиная с D2

                    # # Обновление формата ячеек на "Номер"
                    # sheet.batch_update([{
                    #     'range': 'D2:D',
                    #     'values': current_values,
                    #     'format': {
                    #         'numberFormat': {
                    #             'type': 'NUMBER',
                    #             'pattern': '0'  # Формат без десятичных знаков
                    #         }
                    #     }
                    # }])

                    # 4) Открываем нужный лист (вкладку) по названию 'DATA'
                    worksheet = client.open_by_url(google_sheet_url).worksheet(list)
                    
                    # 5) Применяем форматирование к диапазону D2:D
                    worksheet.format(
                        "D2:D",  # Форматируем весь столбец D начиная со 2-й строки
                        {
                            "numberFormat": {
                                "type": "NUMBER",
                                # Паттерн формата: 0.00 -> число с 2 знаками после запятой
                                # Можно указать "0" для целых или другое (например "0.000")
                                "pattern": "#,##0"
                            }
                        }
                    )

            except Exception as e:
                print(f"Ошибка в процессе обновления для {account['username']}: {e}")

def add_columns_to_instagram_sheets_v1():
    """
    Updates the text in cell H1 to 'music info' for each user's Google Sheet.
    """
    with open("db/main/db.json", "r", encoding="utf-8") as file:
        table_list = json.load(file)
    """
    Adds two columns after column S in the INSTAGRAM sheet for each account in table_list.
    """
    for index, account in enumerate(table_list["accounts"]):
        if index == 0:
            try:
                # Set up Google Sheets API credentials
                scope = [
                    "https://spreadsheets.google.com/feeds",
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive.file",
                    "https://www.googleapis.com/auth/drive"
                ]
                credentials = ServiceAccountCredentials.from_json_keyfile_name(
                    'static/reelstranscription-a94a4b07252e.json', scope)
                client = gspread.authorize(credentials)

                # Open the Google Sheet
                google_sheet_url = f'https://docs.google.com/spreadsheets/d/{account["table_id"]}/edit?usp=sharing'
                worksheet = client.open_by_url(google_sheet_url).worksheet('INSTAGRAM')

                # Добавление двух столбцов после столбца S (19-й индекс)
                worksheet.insert_cols([[], []], 20)  # Вставляем два пустых столбца
                print("столбцы добавлены")

            except Exception as e:
                print(f"Ошибка в процессе добавления столбцов для {account['username']}: {e}")





def add_columns_to_instagram_sheets():
    """
    Finds the 'duration, sec' column, adds two new columns after it,
    and sets up formulas and dropdowns as specified.
    """
    with open("db/main/db.json", "r", encoding="utf-8") as file:
        table_list = json.load(file)

    for index, account in enumerate(table_list["accounts"]):
        if index == 0:
            try:
                # Set up Google Sheets API credentials
                scope = [
                    "https://spreadsheets.google.com/feeds",
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive.file",
                    "https://www.googleapis.com/auth/drive"
                ]
                credentials = ServiceAccountCredentials.from_json_keyfile_name(
                    'static/reelstranscription-a94a4b07252e.json', scope)
                client = gspread.authorize(credentials)

                # Open the Google Sheet
                google_sheet_url = f'https://docs.google.com/spreadsheets/d/{account["table_id"]}/edit?usp=sharing'
                worksheet = client.open_by_url(google_sheet_url).worksheet('INSTAGRAM')

                # Find the column index of 'duration, sec'
                header_row = worksheet.row_values(1)
                duration_col_index = header_row.index('duration, sec') + 1  # +1 because gspread uses 1-based indexing

                # Insert two new columns after 'duration, sec'
                worksheet.insert_cols([['Вирусность']], duration_col_index + 1)  # Добавляем первый столбец
                worksheet.insert_cols([['Вовлеченность']], duration_col_index + 2)  # Добавляем второй столбец

                # Add formula to row 3 in 'Вирусность' column
                formula = '=IF(COUNTIF(F:F, F2)<4, "NO DATA", IF(T2<-0.1, "LOW", IF(T2<= 0.1, "AVERAGE", IF(T2<=0.4, "GOOD", IF(T2<=0.8, "BEST", "VIRUS")))))'
                worksheet.update_cell(3, duration_col_index + 1, formula)

                # Create dropdown in row 4 in 'Вирусность' column
                dropdown_values = ['NO DATA', 'LOW', 'AVERAGE', 'GOOD', 'BEST', 'VIRUS']
                dropdown_colors = ['#3d3d3d', '#b10202', '#ffc8aa', '#ffe5a0', '#d4edbc', '#5a3286']

                # Create data validation rule
                validation_rule = {
                    "setDataValidation": {
                        "range": {
                            "sheetId": worksheet.id,
                            "startRowIndex": 3,  # 4th row (0-based index)
                            "endRowIndex": 4,
                            "startColumnIndex": duration_col_index,  # 'Вирусность' column (0-based index)
                            "endColumnIndex": duration_col_index + 1
                        },
                        "rule": {
                            "condition": {
                                "type": "ONE_OF_LIST",
                                "values": [{"userEnteredValue": value} for value in dropdown_values]
                            },
                            "showCustomUi": True,
                            "strict": True
                        }
                    }
                }

                # Apply data validation
                worksheet.spreadsheet.batch_update({"requests": [validation_rule]})

                # Set background colors for dropdown values
                for i, color in enumerate(dropdown_colors):
                    worksheet.format_cell(4, duration_col_index + 1, {
                        "backgroundColor": {
                            "red": int(color[1:3], 16) / 255,
                            "green": int(color[3:5], 16) / 255,
                            "blue": int(color[5:7], 16) / 255
                        }
                    })

                print("Столбцы, формулы и выпадающий список добавлены")

            except Exception as e:
                print(f"Ошибка в процессе добавления столбцов для {account['username']}: {e}")

def add_new_row_with_values_and_update_filter():
    """
    Вставляет новую строку между первой и второй строками Google Sheet,
    заполняет столбцы "Вирусность" значением "NO DATA" и "Вовлеченность" значением "LOW ER",
    затем обновляет базовый фильтр, чтобы охватывать всю таблицу.
    """


    with open("db/main/db.json", "r", encoding="utf-8") as file:
        table_list = json.load(file)

    for index, account in enumerate(table_list["accounts"]):
        if index == 0:
            try:
                # Настройка авторизации для Google Sheets API
                scope = [
                    "https://spreadsheets.google.com/feeds",
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive.file",
                    "https://www.googleapis.com/auth/drive"
                ]
                credentials = ServiceAccountCredentials.from_json_keyfile_name(
                    'static/reelstranscription-a94a4b07252e.json', scope)
                client = gspread.authorize(credentials)

                # Открываем нужную таблицу и лист
                google_sheet_url = f'https://docs.google.com/spreadsheets/d/{account["table_id"]}/edit?usp=sharing'
                worksheet = client.open_by_url(google_sheet_url).worksheet('TEST_FILTER')
                sheet_id = worksheet.id

                # Получаем заголовок из первой строки и определяем количество столбцов
                header_row = worksheet.row_values(1)
                num_columns = len(header_row)

                # Находим индексы столбцов "Вирусность" и "Вовлеченность"
                try:
                    virus_index = header_row.index("Вирусность")  # 0-based индекс
                except ValueError:
                    raise Exception("Столбец 'Вирусность' не найден в заголовке")

                try:
                    engagement_index = header_row.index("Вовлеченность")
                except ValueError:
                    raise Exception("Столбец 'Вовлеченность' не найден в заголовке")
                try:
                    status_index = header_row.index("status")
                except ValueError:
                    raise Exception("Столбец 'Вовлеченность' не найден в заголовке")

                # Формируем новую строку с пустыми значениями
                new_row = ["" for _ in range(num_columns)]
                new_row[virus_index] = "NO DATA"
                new_row[engagement_index] = "LOW ER"
                new_row[status_index] = "---"

                # Вставляем новую строку на позицию 2 (между заголовком и предыдущей второй строкой)
                worksheet.insert_row(new_row, 3, value_input_option='USER_ENTERED')
                spreadsheet_id = account['table_id']




                #FILTER

                # 3️⃣ Считываем текущие фильтры
                service = build('sheets', 'v4', credentials=credentials)
                spreadsheet = service.spreadsheets().get(spreadsheetId=account["table_id"]).execute()
                sheet_info = next((s for s in spreadsheet['sheets'] if s['properties']['sheetId'] == sheet_id), None)

                if not sheet_info or "basicFilter" not in sheet_info:
                    print("⚠️ Базовый фильтр не найден. Пропускаем обновление.")
                    continue

                existing_filter = sheet_info["basicFilter"]
                print("🎯 Текущие фильтры перед обновлением:")
                print(json.dumps(existing_filter, indent=4, ensure_ascii=False))

                # 4️⃣ Сохраняем значения только для фильтруемых колонок
                criteria_columns = list(existing_filter.get("criteria", {}).keys())
                column_value_map = {}

                # Загружаем только нужные диапазоны для этих колонок
                ranges = [f"'TEST_FILTER'!{chr(65 + int(col))}2:{chr(65 + int(col))}" for col in criteria_columns]
                result = service.spreadsheets().values().batchGet(
                    spreadsheetId=account["table_id"],
                    ranges=ranges
                ).execute()

                # Обрабатываем результат
                for idx, col in enumerate(criteria_columns):
                    column_values = result.get("valueRanges", [])[idx].get("values", [])
                    # Извлекаем уникальные значения, ограничиваем вывод
                    unique_values = {val[0] for val in column_values if val}
                    column_value_map[int(col)] = unique_values
                    print(f"📊 Колонка {col}: {list(unique_values)[:5]}... (показано 5 из {len(unique_values)})")

                # 5️⃣ Отключаем фильтры
                service.spreadsheets().batchUpdate(
                    spreadsheetId=account["table_id"],
                    body={"requests": [{"clearBasicFilter": {"sheetId": sheet_id}}]}
                ).execute()
                print("🚨 Фильтры отключены.")

                # 6️⃣ Добавляем новую строку
                header_row = worksheet.row_values(1)
                new_row = ["" for _ in range(len(header_row))]
                try:
                    virus_index = header_row.index("Вирусность")
                    engagement_index = header_row.index("Вовлеченность")
                    new_row[virus_index] = "NO DATA"
                    new_row[engagement_index] = "LOW ER"
                except ValueError:
                    print("⚠️ Столбцы 'Вирусность' или 'Вовлеченность' не найдены.")




                worksheet.insert_row(new_row, 3, value_input_option='USER_ENTERED')
                print("✅ Новая строка добавлена.")




                # 7️⃣ Обновляем диапазон фильтра
                existing_filter["range"]["endRowIndex"] = worksheet.row_count

                # 8️⃣ Восстанавливаем скрытые значения
                if "criteria" in existing_filter:
                    for col_idx, criteria in existing_filter["criteria"].items():
                        if "hiddenValues" in criteria:
                            current_hidden = set(criteria["hiddenValues"])
                            possible_values = column_value_map.get(int(col_idx), set())
                            # Добавляем только те скрытые значения, которые существуют
                            updated_hidden = current_hidden.intersection(possible_values)
                            existing_filter["criteria"][col_idx]["hiddenValues"] = list(updated_hidden)

                # 9️⃣ Восстанавливаем фильтры
                service.spreadsheets().batchUpdate(
                    spreadsheetId=account["table_id"],
                    body={"requests": [{"setBasicFilter": {"filter": existing_filter}}]}
                ).execute()
                print("🔄 Фильтры восстановлены с обновлённым диапазоном.")





            except Exception as e:
                print(f"Ошибка при выполнении операции для {account['username']}: {e}")

add_new_row_with_values_and_update_filter()

#add_columns_to_instagram_sheets()
# Пример вызова функции
#update_info_in_sheets()

🎯 Текущие фильтры перед обновлением:
{
    "range": {
        "sheetId": 821187531,
        "startRowIndex": 0,
        "endRowIndex": 2848,
        "startColumnIndex": 0,
        "endColumnIndex": 38
    },
    "criteria": {
        "18": {},
        "19": {
            "hiddenValues": [
                "GOOD",
                "NO DATA"
            ]
        },
        "20": {},
        "24": {
            "hiddenValues": [
                "",
                "---",
                "checked",
                "done",
                "NO"
            ]
        }
    },
    "filterSpecs": [
        {
            "columnIndex": 19,
            "filterCriteria": {
                "hiddenValues": [
                    "GOOD",
                    "NO DATA"
                ]
            }
        },
        {
            "columnIndex": 20,
            "filterCriteria": {}
        },
        {
            "columnIndex": 18,
            "filterCriteria": {}
        },
        {
            "col

In [12]:
pip install google-api-python-client


     --------------------------------------- 12.9/12.9 MB 11.9 MB/s eta 0:00:00
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
     -------------------------------------- 160.1/160.1 kB 9.4 MB/s eta 0:00:00
     -------------------------------------- 165.0/165.0 kB 9.7 MB/s eta 0:00:00
     ---------------------------------------- 50.2/50.2 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def main():
    # 1) Указываем скоупы: работа с Google Sheets и Drive.
    scopes = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
    ]

    # 2) Авторизуемся с помощью ключей сервис-аккаунта
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        'credentials.json',  # путь к файлу с ключами
        scopes
    )
    client = gspread.authorize(creds)

    # 3) Открываем нужную гугл-таблицу по названию или по spreadsheet key
    spreadsheet = client.open("Название вашей таблицы")
    
    # 4) Открываем нужный лист (вкладку) по названию 'DATA'
    worksheet = spreadsheet.worksheet("DATA")
    
    # 5) Применяем форматирование к диапазону D2:D
    worksheet.format(
        "D2:D",  # Форматируем весь столбец D начиная со 2-й строки
        {
            "numberFormat": {
                "type": "NUMBER",
                # Паттерн формата: 0.00 -> число с 2 знаками после запятой
                # Можно указать "0" для целых или другое (например "0.000")
                "pattern": "0.00"
            }
        }
    )

    print("Форматирование выполнено успешно!")

if __name__ == "__main__":
    main()

In [15]:
import pandas as pd

def load_json_to_dataframe(file_path):
    return pd.read_json(file_path)

def create_table(json_files):
    # Load all JSON data into DataFrames and sort by 'video'
    dataframes = [load_json_to_dataframe(file).sort_values(by='video').reset_index(drop=True) for file in json_files]

    # Check if all DataFrames have the same number of elements
    num_elements = len(dataframes[0])
    if not all(len(df) == num_elements for df in dataframes):
        raise ValueError("All JSON files must have the same number of elements")

    # Combine DataFrames
    combined_df = pd.DataFrame({
        'id': range(num_elements),  # Row number
        'videopath': dataframes[0]['video']  # Use videopath from the first file only
    })

    # Add transcriptions from each file
    for i, df in enumerate(dataframes):
        combined_df[f'transcription_{i}'] = df['transcription']

    return combined_df

# Main execution
json_files = [
    'reels/test/reels_transcriptions_thread_10.json',
    'reels/test/reels_transcriptions_thread_01.json',
    'reels/test/reels_transcriptions_thread_06.json',
    'reels/test/reels_transcriptions_1_base.json'
]

combined_df = create_table(json_files)
pd.set_option('display.max_colwidth', None)
combined_df  # Display the DataFrame as a table in the notebook

ValueError: Trailing data